In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import mne_connectivity
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 
import itertools

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

In [2]:
sub_list = ['BJH021', 'BJH025', 'BJH016', 'SLCH002', 'BJH026', 'BJH027', 'BJH029', 'BJH039', 'BJH041', 'LL10', 'LL12', 'LL13', 'LL14', 'LL17', 'LL19']


In [3]:
for subject in sub_list:
    # prep fodlers and lod channels, times
    raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
    preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

    sub_times = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/{subject}_times_noghost.npy')
    sub_channels = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/{subject}_channel_names_noghost.npy')

    with open('/home/brooke/pacman/preprocessing/scripts/roi.py') as file:
        exec(file.read())
    ROIs = ROIs[subject]

    # primary ROI
    hc_list = []
    hc_indices = []
    hc_names = []
    ofc_list = []
    ofc_indices = []
    ofc_names = []
    amyg_list = []
    amyg_names = [] 
    amyg_indices = []
    cing_list = []
    cing_names = [] 
    cing_indices = []

    # control ROI
    insula_list = []
    insula_names = []  
    insula_indices = []
    dlpfc_list = []
    dlpfc_names = []  
    dlpfc_indices = []
    ec_list = []
    ec_names = []  
    ec_indices = []

    # exclude bad ROI from list
    pairs_name = [ch.split('-') for ch in sub_channels]
    bidx = 0 # since the bad channels are already removed

    # sort ROI into lists
    for ix in range(0, len(pairs_name)):
        if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
            hc_list.append(sub_channels[ix + bidx])
            hc_names.append(pairs_name[ix])
            hc_indices.append(ix)
        if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
            ofc_list.append(sub_channels[ix + bidx])
            ofc_names.append(pairs_name[ix])
            ofc_indices.append(ix)
        if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
            amyg_list.append(sub_channels[ix + bidx])       
            amyg_names.append(pairs_name[ix])
            amyg_indices.append(ix)
        if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
            cing_list.append(sub_channels[ix + bidx])       
            cing_names.append(pairs_name[ix])
            cing_indices.append(ix)
            
        # control roi
        if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
            insula_list.append(sub_channels[ix + bidx])       
            insula_names.append(pairs_name[ix])
            insula_indices.append(ix)
        if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
            dlpfc_list.append(sub_channels[ix + bidx])       
            dlpfc_names.append(pairs_name[ix])
            dlpfc_indices.append(ix)
        if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
            ec_list.append(sub_channels[ix + bidx])       
            ec_names.append(pairs_name[ix])
            ec_indices.append(ix)    

    ## Load true data ##
    imcoh = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/{subject}_imcoh_noghost.npy')
    ppc = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/{subject}_ppc_noghost.npy')
    pli = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/{subject}_pli_noghost.npy')
    pairs = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/{subject}_pairs_noghost.npy')

    ## Load shuffled data ##
    imcoh_permutations = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/perms/{subject}_imcoh_perm.npy')
    ppc_permutations = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/perms/{subject}_ppc_perm.npy')
    pli_permutations = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/perms/{subject}_pli_perm.npy')
    pairs_perm = np.load(f'/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/perms/{subject}_pairs.npy')   

    ## reshape shuffled data ##
    if(imcoh.shape[1] > 2000):
        imcoh = imcoh[:, ::5]
        ppc = ppc[:, ::5]
        pli = pli[:, ::5]
        sub_times = sub_times[::5]

        imcoh_permutations = imcoh_permutations[:, ::5]
        ppc_permutations = ppc_permutations[:, ::5]
        pli_permutations = pli_permutations[:, ::5]
    imcoh_permutations = np.reshape(imcoh_permutations,(imcoh.shape[0], 1000, imcoh.shape[1]))
    ppc_permutations = np.reshape(ppc_permutations,(ppc.shape[0], 1000, ppc.shape[1]))
    pli_permutations = np.reshape(pli_permutations,(pli.shape[0], 1000, pli.shape[1]))   

    ## get pairs names for dataframes ##
    assert(np.array_equal(pairs_perm,pairs)) # must be true for the rest of the code to work
    pairs_tested = [ f"{sub_channels[x]}_to_{sub_channels[y]}" for x, y in zip(pairs_perm[0], pairs_perm[1])]
    first_pairs = [ f"{sub_channels[x]}" for x in pairs_perm[0]]
    second_pairs = [ f"{sub_channels[y]}" for y in pairs_perm[1]]
    first_roi = ['hc' if x in hc_list else 'ofc' if x in ofc_list else 'amyg' if x in amyg_list else 'insula' if x in insula_list else 'cing' if x in cing_list else 'dlpfc' if x in dlpfc_list else 'other' for x in first_pairs]
    second_roi = ['hc' if x in hc_list else 'ofc' if x in ofc_list else 'amyg' if x in amyg_list else 'insula' if x in insula_list else 'cing' if x in cing_list else 'dlpfc' if x in dlpfc_list else 'other' for x in second_pairs]

    ## calculate p values ##
    imcoh_pvalues = np.zeros((imcoh.shape[0], imcoh.shape[1]))
    ppc_pvalues = np.zeros((imcoh.shape[0], imcoh.shape[1]))
    pli_pvalues = np.zeros((imcoh.shape[0], imcoh.shape[1]))

    for pair in range(0, imcoh.shape[0]):

        # calculate imcoh p-values
        null_greater_than_imcoh = np.abs(imcoh[pair, :]) <= np.abs(imcoh_permutations[pair, :, :]) 
        percent_null_greater_than_imcoh = np.sum(null_greater_than_imcoh, axis = 0)/1000
        imcoh_pvalues[pair, :] =  percent_null_greater_than_imcoh

        # calculate ppc p-values
        null_greater_than_ppc = ppc[pair, :] <= ppc_permutations[pair, :, :]
        percent_null_greater_than_ppc = np.sum(null_greater_than_ppc, axis = 0)/1000
        ppc_pvalues[pair, :] = percent_null_greater_than_ppc

        # calculate pli p-values
        null_greater_than_pli = pli[pair, :] <= pli_permutations[pair, :, :]
        percent_null_greater_than_pli = np.sum(null_greater_than_pli, axis = 0)/1000
        pli_pvalues[pair, :] = percent_null_greater_than_pli


    ## create data frames for connectivity measures and p-values ##
    ## IMCOH DF ##
    imcoh_df = pd.DataFrame(imcoh, columns = sub_times)
    imcoh_df['pairs'] = pairs_tested
    imcoh_df['first_region'] = first_roi
    imcoh_df['second_region'] = second_roi
    imcoh_df = imcoh_df.melt(id_vars = ['pairs', 'first_region', 'second_region'], var_name='time', value_name='connectivity')
    imcoh_df['metric'] = 'Imaginary Coherence'

    ## PPC DF ##
    ppc_df = pd.DataFrame(ppc, columns = sub_times)
    ppc_df['pairs'] = pairs_tested
    ppc_df['first_region'] = first_roi
    ppc_df['second_region'] = second_roi
    ppc_df = ppc_df.melt(id_vars = ['pairs', 'first_region', 'second_region'], var_name='time', value_name='connectivity')
    ppc_df['metric'] = 'Pairwise Phase Consistency'

    ## PLI DF ##
    pli_df = pd.DataFrame(pli, columns = sub_times)
    pli_df['pairs'] = pairs_tested
    pli_df['first_region'] = first_roi
    pli_df['second_region'] = second_roi
    pli_df = pli_df.melt(id_vars = ['pairs', 'first_region', 'second_region'], var_name='time', value_name='connectivity')
    pli_df['metric'] = 'Phase Lag Index'


    ## Imcoh P-Value DF ##
    pval_imcoh_df = pd.DataFrame(imcoh_pvalues, columns =sub_times)
    pval_imcoh_df['pairs'] = pairs_tested
    pval_imcoh_df['first_region'] = first_roi
    pval_imcoh_df['second_region'] = second_roi
    pval_imcoh_df = pval_imcoh_df.melt(id_vars = ['pairs', 'first_region', 'second_region'], var_name='time', value_name='percent_sig')
    pval_imcoh_df['metric'] = 'Imaginary Coherence'

    ## PPC P-Value DF ##
    pval_ppc_df = pd.DataFrame(ppc_pvalues, columns =sub_times)
    pval_ppc_df['pairs'] = pairs_tested
    pval_ppc_df['first_region'] = first_roi
    pval_ppc_df['second_region'] = second_roi
    pval_ppc_df = pval_ppc_df.melt(id_vars = ['pairs', 'first_region', 'second_region'], var_name='time', value_name='percent_sig')
    pval_ppc_df['metric'] = 'Pairwise Phase Consistency'

    ## PLI P-Value DF ##
    pval_pli_df = pd.DataFrame(pli_pvalues, columns = sub_times)
    pval_pli_df['pairs'] = pairs_tested
    pval_pli_df['first_region'] = first_roi
    pval_pli_df['second_region'] = second_roi
    pval_pli_df = pval_pli_df.melt(id_vars = ['pairs', 'first_region', 'second_region'], var_name='time', value_name='percent_sig')
    pval_pli_df['metric'] = 'Phase Lag Index'

    ## Combine ##
    imcoh_final_df = pd.merge(imcoh_df, pval_imcoh_df, how = 'outer')
    ppc_final_df = pd.merge(ppc_df, pval_ppc_df, how = 'outer')
    pli_final_df = pd.merge(pli_df, pval_pli_df, how = 'outer')

    final_sub_df = pd.concat([imcoh_final_df, ppc_final_df, pli_final_df])
    final_sub_df['subject'] = subject

    ## Bind ##
    if subject == sub_list[0]:
        final_df = final_sub_df
    else:
        final_df = pd.concat([final_df, final_sub_df])




In [4]:
final_df

,pairs,first_region,second_region,time,connectivity,metric,percent_sig,subject
0,A1-A2_to_A10-A11,amyg,insula,-2.5,0.102424,Imaginary Coherence,0.021,BJH021
1,A1-A2_to_A10-A11,amyg,insula,-2.49,0.098080,Imaginary Coherence,0.024,BJH021
2,A1-A2_to_A10-A11,amyg,insula,-2.48,0.093736,Imaginary Coherence,0.026,BJH021
3,A1-A2_to_A10-A11,amyg,insula,-2.47,0.089439,Imaginary Coherence,0.036,BJH021
4,A1-A2_to_A10-A11,amyg,insula,-2.46,0.085242,Imaginary Coherence,0.043,BJH021
...,...,...,...,...,...,...,...,...
69250,ROF3-ROF4_to_ROF9-ROF10,ofc,dlpfc,2.460938,0.042749,Phase Lag Index,0.074,LL19
69251,ROF3-ROF4_to_ROF9-ROF10,ofc,dlpfc,2.470703,0.036796,Phase Lag Index,0.096,LL19
69252,ROF3-ROF4_to_ROF9-ROF10,ofc,dlpfc,2.480469,0.031891,Phase Lag Index,0.113,LL19
69253,ROF3-ROF4_to_ROF9-ROF10,ofc,dlpfc,2.490234,0.028628,Phase Lag Index,0.133,LL19


In [7]:
pd.DataFrame.to_csv(final_df, '/home/brooke/pacman/connectivity/ieeg/imcoh_ppc_pli/combined_noghost_connectivity.csv', index = False)